### Manipulation of Peaks de novo results of ETNP 2017 P2 samples LC-MS/MS data using python.

Starting with:

    Peaks de novo results (.csv) of PTM-optimized database searches

Goal:

    Files with stripped (no PTMs) peptide lists and
    Columns with #'s of each modification in every sequence
    Column with stripped peptide lengths (# amino acids)
    
### To use for a different file:

#### 1. Change the input file name in *IN 4*
#### 2. Use 'find + replace' (Esc + F) to replace the running # (e.g., 233) for another
#### 3. Update the NAAF factor calculated in *IN 6* into *IN 7*

We don't have technical duplicates here, sadly, unlike the MED4 Pro samples. I exported PeaksDN search results CSVs into my ETNP 2017 git repo:

In [1]:
cd /home/millieginty/Documents/git-repos/2017-etnp/data/pro2020/ETNP-SKQ17/PEAKS-PTMopt/

/home/millieginty/Documents/git-repos/2017-etnp/data/pro2020/ETNP-SKQ17/PEAKS-PTMopt


In [2]:
# LIBRARIES
#import pandas library for working with tabular data
import os
os.getcwd()
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.stats import kde
#import regular expresson (regex)
import re
#check pandas version
pd.__version__

'1.0.5'

## 273: 965 m +P filtered on 0.3 um GF-75

In [3]:
# read the CSV into a dataframe using the pandas read_csv function
peaks273_50 = pd.read_csv("/home/millieginty/Documents/git-repos/2017-etnp/data/pro2020/ETNP-SKQ17/PEAKS-PTMopt/ETNP-SKQ17-273-965m-trap_DN50.csv")

# also make a dataframe but keep only peptides >80% ALC
peaks273_80 = peaks273_50.loc[peaks273_50['ALC (%)'] >= 80].copy()

# how many de novo sequence candidates >50% ALC?
print("# redundant Peaks peptides >50% ALC in dataframe", len(peaks273_50))
# how many de novo sequence candidates >80ALC?
print("# redundant Peaks peptides >80% ALC in dataframe", len(peaks273_80))

#look at the dataframe
peaks273_50.head()

# redundant Peaks peptides >50% ALC in dataframe 8727
# redundant Peaks peptides >80% ALC in dataframe 1086


,Fraction,Scan,Source File,Peptide,Tag Length,ALC (%),length,m/z,z,RT,Area,Mass,ppm,PTM,local confidence (%),tag (>=0%),mode
0,6,12978,20181003_4-19_273_965m_top.raw,LLYEEEK,7,97,7,462.2459,2,40.82,2450000.0,922.4647,13.6,NaN,99 99 97 99 99 98 95,LLYEEEK,CID
1,6,6251,20181003_4-19_273_965m_top.raw,EEDLER,6,97,6,395.6828,2,22.37,7190000.0,789.3504,0.9,NaN,99 100 97 98 99 92,EEDLER,CID
2,6,26188,20181003_4-19_273_965m_top.raw,LLEWLK,6,97,6,401.2476,2,69.91,927000.0,800.4796,1.3,NaN,98 97 98 95 98 96,LLEWLK,CID
3,6,8671,20181003_4-19_273_965m_top.raw,TEAPLNPK,8,97,8,435.2407,2,28.51,41300000.0,868.4654,1.6,NaN,99 100 100 95 97 94 97 95,TEAPLNPK,CID
4,6,6790,20181003_4-19_273_965m_top.raw,LYEETR,6,96,6,405.7037,2,23.57,4990000.0,809.3919,1.2,NaN,97 97 99 99 96 93,LYEETR,CID


In [4]:
# use a count function to enumerate the # of A's (alanines) in each peptide
peaks273_50['A'] = peaks273_50['Peptide'].str.count("A")

# use a count function to enumerate the # of C's (cysteines) in each peptide
peaks273_50['C'] = peaks273_50['Peptide'].str.count("C")

# use a count function to enumerate the # of D's (aspartic acids) in each peptide
peaks273_50['D'] = peaks273_50['Peptide'].str.count("D")

# use a count function to enumerate the # of E's (glutamic acids) in each peptide
peaks273_50['E'] = peaks273_50['Peptide'].str.count("E")

# use a count function to enumerate the # of F's (phenylalanines) in each peptide
peaks273_50['F'] = peaks273_50['Peptide'].str.count("F")

# use a count function to enumerate the # of G's (glycines) in each peptide
peaks273_50['G'] = peaks273_50['Peptide'].str.count("G")

# use a count function to enumerate the # of H's (histidines) in each peptide
peaks273_50['H'] = peaks273_50['Peptide'].str.count("H")

# use a count function to enumerate the # of I's (isoleucines) in each peptide
# in peaks273_50 output, there will be no isoleucines (they're lumped in with leucines)
peaks273_50['I'] = peaks273_50['Peptide'].str.count("I")

# use a count function to enumerate the # of K's (lysines) in each peptide
peaks273_50['K'] = peaks273_50['Peptide'].str.count("K")

# use a count function to enumerate the # of L's (leucines) in each peptide
# also these include the isoleucines
peaks273_50['I/L'] = peaks273_50['Peptide'].str.count("L")

# use a count function to enumerate the # of M's (methionines) in each peptide
peaks273_50['M'] = peaks273_50['Peptide'].str.count("M")

# use a count function to enumerate the # of N's (asparagines) in each peptide
peaks273_50['N'] = peaks273_50['Peptide'].str.count("N")

# use a count function to enumerate the # of P's ([prolines]) in each peptide
peaks273_50['P'] = peaks273_50['Peptide'].str.count("P")

# use a count function to enumerate the # of Q's (glutamines) in each peptide
peaks273_50['Q'] = peaks273_50['Peptide'].str.count("Q")

# use a count function to enumerate the # of R's (arginines) in each peptide
peaks273_50['R'] = peaks273_50['Peptide'].str.count("R")

# use a count function to enumerate the # of S's (serines) in each peptide
peaks273_50['S'] = peaks273_50['Peptide'].str.count("S")

# use a count function to enumerate the # of T's (threonines) in each peptide
peaks273_50['T'] = peaks273_50['Peptide'].str.count("T")

# use a count function to enumerate the # of V's (valines) in each peptide
peaks273_50['V'] = peaks273_50['Peptide'].str.count("V")

# use a count function to enumerate the # of W's (tryptophans) in each peptide
peaks273_50['W'] = peaks273_50['Peptide'].str.count("W")

# use a count function to enumerate the # of Y's (tyrosines) in each peptide
peaks273_50['Y'] = peaks273_50['Peptide'].str.count("Y")

# use a count function to enumerate the # of carbamidomethylated C's in each peptide
peaks273_50['c-carb'] = peaks273_50['Peptide'].str.count("57.02")

# use a count function to enumerate the # of oxidized M's in each peptide
peaks273_50['m-oxid'] = peaks273_50['Peptide'].apply(lambda x: x.count('M(+15.99)'))

# use a lamba function to enumerate the # of deamidated N's in each peptide
peaks273_50['n-deam'] = peaks273_50['Peptide'].apply(lambda x: x.count('N(+.98)'))

# use a count function to enumerate the # of deamidated Q's in each peptide
peaks273_50['q-deam'] = peaks273_50['Peptide'].apply(lambda x: x.count('Q(+.98)'))

# use a count function to enumerate the # of hydroxylated K's in each peptide
peaks273_50['k-hydr'] = peaks273_50['Peptide'].apply(lambda x: x.count('K(+15.99)'))

# use a count function to enumerate the # of hydroxylated P's in each peptide
peaks273_50['p-hydr'] = peaks273_50['Peptide'].apply(lambda x: x.count('P(+15.99)'))

# use a count function to enumerate the # of methylated R's in each peptide
peaks273_50['r-meth'] = peaks273_50['Peptide'].apply(lambda x: x.count('R(+14.02)'))

# create a column with 'stripped' peptide sequences using strip
peaks273_50['stripped peptide'] = peaks273_50['Peptide'].str.replace(r"\(.*\)","")

# add a column with the stripped peptide length (number of AAs)
peaks273_50['stripped length'] = peaks273_50['stripped peptide'].apply(len)

# total the number of modifications in sequence
peaks273_50['ptm-total'] = peaks273_50['c-carb'] + peaks273_50['m-oxid'] + peaks273_50['n-deam'] + peaks273_50['q-deam'] + peaks273_50['k-hydr'] + peaks273_50['p-hydr'] + peaks273_50['r-meth']

# calculate NAAF numerator for each peptide k
peaks273_50['NAAF num.'] = peaks273_50['Area'] / peaks273_50['stripped length']

# write modified dataframe to new csv file
peaks273_50.to_csv("/home/millieginty/Documents/git-repos/2017-etnp/data/pro2020/ETNP-SKQ17/PEAKS-PTMopt/ETNP-SKQ17-273-965m-trap_PTMopt_DN50.csv")


# check out the results
peaks273_50.head()

,Fraction,Scan,Source File,Peptide,Tag Length,ALC (%),length,m/z,z,RT,...,m-oxid,n-deam,q-deam,k-hydr,p-hydr,r-meth,stripped peptide,stripped length,ptm-total,NAAF num.
0,6,12978,20181003_4-19_273_965m_top.raw,LLYEEEK,7,97,7,462.2459,2,40.82,...,0,0,0,0,0,0,LLYEEEK,7,0,3.500000e+05
1,6,6251,20181003_4-19_273_965m_top.raw,EEDLER,6,97,6,395.6828,2,22.37,...,0,0,0,0,0,0,EEDLER,6,0,1.198333e+06
2,6,26188,20181003_4-19_273_965m_top.raw,LLEWLK,6,97,6,401.2476,2,69.91,...,0,0,0,0,0,0,LLEWLK,6,0,1.545000e+05
3,6,8671,20181003_4-19_273_965m_top.raw,TEAPLNPK,8,97,8,435.2407,2,28.51,...,0,0,0,0,0,0,TEAPLNPK,8,0,5.162500e+06
4,6,6790,20181003_4-19_273_965m_top.raw,LYEETR,6,96,6,405.7037,2,23.57,...,0,0,0,0,0,0,LYEETR,6,0,8.316667e+05


In [5]:
# made a new dataframe that contains the sums of certain columns in the modified
# peptide dataframe above (for >50% ALC)

index = ['sample total']

data = {'A': peaks273_50['A'].sum(),
        'C': peaks273_50['C'].sum(),
        'D': peaks273_50['D'].sum(),
        'E': peaks273_50['E'].sum(),
        'F': peaks273_50['F'].sum(),
        'G': peaks273_50['G'].sum(),
        'H': peaks273_50['H'].sum(),
        'I': peaks273_50['I'].sum(),
        'K': peaks273_50['K'].sum(),
        'I/L': peaks273_50['I/L'].sum(),
        'M': peaks273_50['M'].sum(),
        'N': peaks273_50['N'].sum(),
        'P': peaks273_50['P'].sum(),
        'Q': peaks273_50['Q'].sum(),
        'R': peaks273_50['R'].sum(),
        'S': peaks273_50['S'].sum(),
        'T': peaks273_50['T'].sum(),
        'V': peaks273_50['V'].sum(),
        'W': peaks273_50['W'].sum(),
        'Y': peaks273_50['Y'].sum(),
        'c-carb': peaks273_50['c-carb'].sum(),
        'm-oxid': peaks273_50['m-oxid'].sum(),
        'n-deam': peaks273_50['n-deam'].sum(),
        'q-deam': peaks273_50['q-deam'].sum(),
        'k-hydr': peaks273_50['k-hydr'].sum(),
        'p-hydr': peaks273_50['p-hydr'].sum(),
        'r-meth': peaks273_50['r-meth'].sum(),
        'Total area': peaks273_50['Area'].sum(),
        'Total length': peaks273_50['stripped length'].sum()
       }

totalpeaks273_50 = pd.DataFrame(data, columns=['A', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'K', 'I/L', 'M', 'N', 'P', 'Q', 'R', 'S', 'T', 'V', 'W', 'Y', 'c-carb', 'm-oxid', 'n-deam', 'q-deam', 'k-hydr', 'p-hydr', 'r-meth', 'Total area', 'Total length'], index=index)

# calculate percentage of C's with carb (should be 1.0)
totalpeaks273_50['% C w/ carb'] = totalpeaks273_50['c-carb'] / totalpeaks273_50['C'] 

# calculate percentage of M's that are oxidized
totalpeaks273_50['% M w/ oxid'] = totalpeaks273_50['m-oxid'] / totalpeaks273_50['M'] 

# calculate percentage of N's that are deamidated
totalpeaks273_50['% N w/ deam'] = totalpeaks273_50['n-deam'] / totalpeaks273_50['N'] 

# calculate percentage of Q's that are deamidated
totalpeaks273_50['% Q w/ deam'] = totalpeaks273_50['q-deam'] / totalpeaks273_50['Q'] 

# calculate percentage of K's that are hydroxylated
totalpeaks273_50['% K w/ hydr'] = totalpeaks273_50['k-hydr'] / totalpeaks273_50['K'] 

# calculate percentage of P's that are hydroxylated
totalpeaks273_50['% P w/ hydr'] = totalpeaks273_50['p-hydr'] / totalpeaks273_50['K'] 

# calculate percentage of R's that are methylated
totalpeaks273_50['% R w/ meth'] = totalpeaks273_50['r-meth'] / totalpeaks273_50['R'] 

# calculate NAAF denominator for all peptides in dataset i
totalpeaks273_50['NAAF denom.'] = totalpeaks273_50['Total area'] / totalpeaks273_50['Total length']

# write modified dataframe to new txt file
totalpeaks273_50.to_csv("/home/millieginty/Documents/git-repos/2017-etnp/data/pro2020/ETNP-SKQ17/PEAKS-PTMopt/ETNP-SKQ17-273-965m-trap_PTMopt_DN50_totals.csv")

totalpeaks273_50.head()

,A,C,D,E,F,G,H,I,K,I/L,...,Total area,Total length,% C w/ carb,% M w/ oxid,% N w/ deam,% Q w/ deam,% K w/ hydr,% P w/ hydr,% R w/ meth,NAAF denom.
sample total,4262,1332,3400,4505,2686,2827,2200,0,9404,6117,...,1.527343e+10,60575,1.0,0.368797,0.217347,0.02902,0.163654,0.294662,0.482649,252140.749952


In [12]:
# use the calculated NAAF factor (in totalpeaks273 dataframe, above) to caluclate the NAAF 
# NAAF: normalized normalized area abundance factor

# don't have to worry here about DECOY hits messing with Area totals
# but we will with Comet results

NAAF50 = 252140.749952

# use NAAF >50% ALC to get NAAF
peaks273_50['NAAF factor'] = (peaks273_50['NAAF num.'])/NAAF50

# make a dataframe that contains only what we need: sequences, AAs, PTMs
peaks273_AA50 = peaks273_50[['stripped peptide', 'NAAF factor', 'A', 'C', 'D', 'E', 'F', 'G', 'H', 'K', 'I/L', 'M', 'N', 'P', 'Q', 'R', 'S', 'T', 'V', 'W', 'Y', 'c-carb', 'm-oxid', 'n-deam', 'q-deam', 'k-hydr', 'p-hydr', 'r-meth']].copy()

# multiply the NAAF50 factor by the AAs to normalize its abundance by peak area and peptide length

peaks273_AA50['A-NAAF50'] = peaks273_AA50['A'] * peaks273_50['NAAF factor']
peaks273_AA50['C-NAAF50'] = peaks273_AA50['C'] * peaks273_50['NAAF factor']
peaks273_AA50['D-NAAF50'] = peaks273_AA50['D'] * peaks273_50['NAAF factor']
peaks273_AA50['E-NAAF50'] = peaks273_AA50['E'] * peaks273_50['NAAF factor']
peaks273_AA50['F-NAAF50'] = peaks273_AA50['F'] * peaks273_50['NAAF factor']
peaks273_AA50['G-NAAF50'] = peaks273_AA50['G'] * peaks273_50['NAAF factor']
peaks273_AA50['H-NAAF50'] = peaks273_AA50['H'] * peaks273_50['NAAF factor']
peaks273_AA50['K-NAAF50'] = peaks273_AA50['K'] * peaks273_50['NAAF factor']
peaks273_AA50['I/L-NAAF50'] = peaks273_AA50['I/L'] * peaks273_50['NAAF factor']
peaks273_AA50['M-NAAF50'] = peaks273_AA50['M'] * peaks273_50['NAAF factor']
peaks273_AA50['N-NAAF50'] = peaks273_AA50['N'] * peaks273_50['NAAF factor']
peaks273_AA50['P-NAAF50'] = peaks273_AA50['P'] * peaks273_50['NAAF factor']
peaks273_AA50['Q-NAAF50'] = peaks273_AA50['Q'] * peaks273_50['NAAF factor']
peaks273_AA50['R-NAAF50'] = peaks273_AA50['R'] * peaks273_50['NAAF factor']
peaks273_AA50['S-NAAF50'] = peaks273_AA50['S'] * peaks273_50['NAAF factor']
peaks273_AA50['T-NAAF50'] = peaks273_AA50['T'] * peaks273_50['NAAF factor']
peaks273_AA50['V-NAAF50'] = peaks273_AA50['V'] * peaks273_50['NAAF factor']
peaks273_AA50['W-NAAF50'] = peaks273_AA50['W'] * peaks273_50['NAAF factor']
peaks273_AA50['Y-NAAF50'] = peaks273_AA50['Y'] * peaks273_50['NAAF factor']

# multiply the NAAF50 factor by the PTMs normalize its abundance by peak area and peptide length

peaks273_AA50['ccarb-NAAF50'] = peaks273_AA50['c-carb'] * peaks273_AA50['NAAF factor']
peaks273_AA50['moxid-NAAF50'] = peaks273_AA50['m-oxid'] * peaks273_AA50['NAAF factor']
peaks273_AA50['ndeam-NAAF50'] = peaks273_AA50['n-deam'] * peaks273_AA50['NAAF factor']
peaks273_AA50['qdeam-NAAF50'] = peaks273_AA50['q-deam'] * peaks273_AA50['NAAF factor']
peaks273_AA50['khydr-NAAF50'] = peaks273_AA50['k-hydr'] * peaks273_AA50['NAAF factor']
peaks273_AA50['phydr-NAAF50'] = peaks273_AA50['p-hydr'] * peaks273_AA50['NAAF factor']
peaks273_AA50['rmeth-NAAF50'] = peaks273_AA50['r-meth'] * peaks273_AA50['NAAF factor']

# write the dataframe to a new csv
peaks273_AA50.to_csv("/home/millieginty/Documents/git-repos/2017-etnp/data/pro2020/ETNP-SKQ17/PEAKS-PTMopt/ETNP-SKQ17-273-965m-trap_PTMopt_DN50_AA_NAAF.csv")

peaks273_AA50.head()

,stripped peptide,NAAF factor,A,C,D,E,F,G,H,K,...,V-NAAF50,W-NAAF50,Y-NAAF50,ccarb-NAAF50,moxid-NAAF50,ndeam-NAAF50,qdeam-NAAF50,khydr-NAAF50,phydr-NAAF50,rmeth-NAAF50
0,LLYEEEK,1.388114,0,0,0,3,0,0,0,1,...,0.0,0.000000,1.388114,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,EEDLER,4.752637,0,0,1,3,0,0,0,0,...,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,LLEWLK,0.612753,0,0,0,1,0,0,0,1,...,0.0,0.612753,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,TEAPLNPK,20.474675,1,0,0,1,0,0,0,1,...,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,LYEETR,3.298422,0,0,0,2,0,0,0,0,...,0.0,0.000000,3.298422,0.0,0.0,0.0,0.0,0.0,0.0,0.0


### Same process but for de novo peptide >80 % ALC:

In [7]:
# use a count function to enumerate the # of A's (alanines) in each peptide
peaks273_80['A'] = peaks273_80['Peptide'].str.count("A")

# use a count function to enumerate the # of C's (cysteines) in each peptide
peaks273_80['C'] = peaks273_80['Peptide'].str.count("C")

# use a count function to enumerate the # of D's (aspartic acids) in each peptide
peaks273_80['D'] = peaks273_80['Peptide'].str.count("D")

# use a count function to enumerate the # of E's (glutamic acids) in each peptide
peaks273_80['E'] = peaks273_80['Peptide'].str.count("E")

# use a count function to enumerate the # of F's (phenylalanines) in each peptide
peaks273_80['F'] = peaks273_80['Peptide'].str.count("F")

# use a count function to enumerate the # of G's (glycines) in each peptide
peaks273_80['G'] = peaks273_80['Peptide'].str.count("G")

# use a count function to enumerate the # of H's (histidines) in each peptide
peaks273_80['H'] = peaks273_80['Peptide'].str.count("H")

# use a count function to enumerate the # of I's (isoleucines) in each peptide
# in peaks273_80 output, there will be no isoleucines (they're lumped in with leucines)
peaks273_80['I'] = peaks273_80['Peptide'].str.count("I")

# use a count function to enumerate the # of K's (lysines) in each peptide
peaks273_80['K'] = peaks273_80['Peptide'].str.count("K")

# use a count function to enumerate the # of L's (leucines) in each peptide
# also these include the isoleucines
peaks273_80['I/L'] = peaks273_80['Peptide'].str.count("L")

# use a count function to enumerate the # of M's (methionines) in each peptide
peaks273_80['M'] = peaks273_80['Peptide'].str.count("M")

# use a count function to enumerate the # of N's (asparagines) in each peptide
peaks273_80['N'] = peaks273_80['Peptide'].str.count("N")

# use a count function to enumerate the # of P's ([prolines]) in each peptide
peaks273_80['P'] = peaks273_80['Peptide'].str.count("P")

# use a count function to enumerate the # of Q's (glutamines) in each peptide
peaks273_80['Q'] = peaks273_80['Peptide'].str.count("Q")

# use a count function to enumerate the # of R's (arginines) in each peptide
peaks273_80['R'] = peaks273_80['Peptide'].str.count("R")

# use a count function to enumerate the # of S's (serines) in each peptide
peaks273_80['S'] = peaks273_80['Peptide'].str.count("S")

# use a count function to enumerate the # of T's (threonines) in each peptide
peaks273_80['T'] = peaks273_80['Peptide'].str.count("T")

# use a count function to enumerate the # of V's (valines) in each peptide
peaks273_80['V'] = peaks273_80['Peptide'].str.count("V")

# use a count function to enumerate the # of W's (tryptophans) in each peptide
peaks273_80['W'] = peaks273_80['Peptide'].str.count("W")

# use a count function to enumerate the # of Y's (tyrosines) in each peptide
peaks273_80['Y'] = peaks273_80['Peptide'].str.count("Y")

# use a count function to enumerate the # of carbamidomethylated C's in each peptide
peaks273_80['c-carb'] = peaks273_80['Peptide'].str.count("57.02")

# use a count function to enumerate the # of oxidized M's in each peptide
peaks273_80['m-oxid'] = peaks273_80['Peptide'].apply(lambda x: x.count('M(+15.99)'))

# use a lamba function to enumerate the # of deamidated N's in each peptide
peaks273_80['n-deam'] = peaks273_80['Peptide'].apply(lambda x: x.count('N(+.98)'))

# use a count function to enumerate the # of deamidated Q's in each peptide
peaks273_80['q-deam'] = peaks273_80['Peptide'].apply(lambda x: x.count('Q(+.98)'))

# use a count function to enumerate the # of hydroxylated K's in each peptide
peaks273_80['k-hydr'] = peaks273_80['Peptide'].apply(lambda x: x.count('K(+15.99)'))

# use a count function to enumerate the # of hydroxylated P's in each peptide
peaks273_80['p-hydr'] = peaks273_80['Peptide'].apply(lambda x: x.count('P(+15.99)'))

# use a count function to enumerate the # of methylated R's in each peptide
peaks273_80['r-meth'] = peaks273_80['Peptide'].apply(lambda x: x.count('R(+14.02)'))

# create a column with 'stripped' peptide sequences using strip
peaks273_80['stripped peptide'] = peaks273_80['Peptide'].str.replace(r"\(.*\)","")

# add a column with the stripped peptide length (number of AAs)
peaks273_80['stripped length'] = peaks273_80['stripped peptide'].apply(len)

# total the number of modifications in sequence
peaks273_80['ptm-total'] = peaks273_80['c-carb'] + peaks273_80['m-oxid'] + peaks273_80['n-deam'] + peaks273_80['q-deam'] + peaks273_80['k-hydr'] + peaks273_80['p-hydr'] + peaks273_80['r-meth']

# calculate NAAF numerator for each peptide k
peaks273_80['NAAF num.'] = peaks273_80['Area'] / peaks273_80['stripped length']

# write modified dataframe to new csv file
peaks273_80.to_csv("/home/millieginty/Documents/git-repos/2017-etnp/data/pro2020/ETNP-SKQ17/PEAKS-PTMopt/ETNP-SKQ17-273-965m-trap_PTMopt_DN80.csv")

# check out the results
peaks273_80.head()

,Fraction,Scan,Source File,Peptide,Tag Length,ALC (%),length,m/z,z,RT,...,m-oxid,n-deam,q-deam,k-hydr,p-hydr,r-meth,stripped peptide,stripped length,ptm-total,NAAF num.
0,6,12978,20181003_4-19_273_965m_top.raw,LLYEEEK,7,97,7,462.2459,2,40.82,...,0,0,0,0,0,0,LLYEEEK,7,0,3.500000e+05
1,6,6251,20181003_4-19_273_965m_top.raw,EEDLER,6,97,6,395.6828,2,22.37,...,0,0,0,0,0,0,EEDLER,6,0,1.198333e+06
2,6,26188,20181003_4-19_273_965m_top.raw,LLEWLK,6,97,6,401.2476,2,69.91,...,0,0,0,0,0,0,LLEWLK,6,0,1.545000e+05
3,6,8671,20181003_4-19_273_965m_top.raw,TEAPLNPK,8,97,8,435.2407,2,28.51,...,0,0,0,0,0,0,TEAPLNPK,8,0,5.162500e+06
4,6,6790,20181003_4-19_273_965m_top.raw,LYEETR,6,96,6,405.7037,2,23.57,...,0,0,0,0,0,0,LYEETR,6,0,8.316667e+05


In [8]:
# made a new dataframe that contains the sums of certain columns in modified 
#peptide dataframe above (for >80% ALC)

index = ['sample total']

data = {'A': peaks273_80['A'].sum(),
        'C': peaks273_80['C'].sum(),
        'D': peaks273_80['D'].sum(),
        'E': peaks273_80['E'].sum(),
        'F': peaks273_80['F'].sum(),
        'G': peaks273_80['G'].sum(),
        'H': peaks273_80['H'].sum(),
        'I': peaks273_80['I'].sum(),
        'K': peaks273_80['K'].sum(),
        'I/L': peaks273_80['I/L'].sum(),
        'M': peaks273_80['M'].sum(),
        'N': peaks273_80['N'].sum(),
        'P': peaks273_80['P'].sum(),
        'Q': peaks273_80['Q'].sum(),
        'R': peaks273_80['R'].sum(),
        'S': peaks273_80['S'].sum(),
        'T': peaks273_80['T'].sum(),
        'V': peaks273_80['V'].sum(),
        'W': peaks273_80['W'].sum(),
        'Y': peaks273_80['Y'].sum(),
        'c-carb': peaks273_80['c-carb'].sum(),
        'm-oxid': peaks273_80['m-oxid'].sum(),
        'n-deam': peaks273_80['n-deam'].sum(),
        'q-deam': peaks273_80['q-deam'].sum(),
        'k-hydr': peaks273_80['k-hydr'].sum(),
        'p-hydr': peaks273_80['p-hydr'].sum(),
        'r-meth': peaks273_80['r-meth'].sum(),
        'Total area': peaks273_80['Area'].sum(),
        'Total length': peaks273_80['stripped length'].sum()
       }

totalpeaks273_80 = pd.DataFrame(data, columns=['A', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'K', 'I/L', 'M', 'N', 'P', 'Q', 'R', 'S', 'T', 'V', 'W', 'Y', 'c-carb', 'm-oxid', 'n-deam', 'q-deam', 'k-hydr', 'p-hydr', 'r-meth', 'Total area', 'Total length'], index=index)

# calculate percentage of C's with carb (should be 1.0)
totalpeaks273_80['% C w/ carb'] = totalpeaks273_80['c-carb'] / totalpeaks273_80['C'] 

# calculate percentage of M's that are oxidized
totalpeaks273_80['% M w/ oxid'] = totalpeaks273_80['m-oxid'] / totalpeaks273_80['M'] 

# calculate percentage of N's that are deamidated
totalpeaks273_80['% N w/ deam'] = totalpeaks273_80['n-deam'] / totalpeaks273_80['N'] 

# calculate percentage of Q's that are deamidated
totalpeaks273_80['% Q w/ deam'] = totalpeaks273_80['q-deam'] / totalpeaks273_80['Q'] 

# calculate percentage of K's that are hydroxylated
totalpeaks273_80['% K w/ hydr'] = totalpeaks273_80['k-hydr'] / totalpeaks273_80['K'] 

# calculate percentage of P's that are hydroxylated
totalpeaks273_80['% P w/ hydr'] = totalpeaks273_80['p-hydr'] / totalpeaks273_80['K'] 

# calculate percentage of R's that are methylated
totalpeaks273_80['% R w/ meth'] = totalpeaks273_80['r-meth'] / totalpeaks273_80['R'] 

# calculate NAAF denominator for all peptides in dataset i
totalpeaks273_80['NAAF denom.'] = totalpeaks273_80['Total area'] / totalpeaks273_80['Total length']

# write modified dataframe to new txt file
totalpeaks273_80.to_csv("/home/millieginty/Documents/git-repos/2017-etnp/data/pro2020/ETNP-SKQ17/PEAKS-PTMopt/ETNP-SKQ17-273-965m-trap_PTMopt_DN80_totals.csv")

totalpeaks273_80.head()

,A,C,D,E,F,G,H,I,K,I/L,...,Total area,Total length,% C w/ carb,% M w/ oxid,% N w/ deam,% Q w/ deam,% K w/ hydr,% P w/ hydr,% R w/ meth,NAAF denom.
sample total,580,89,491,783,333,336,97,0,902,1122,...,4.628483e+09,7953,1.0,0.526531,0.316667,0.088983,0.09867,0.247228,0.293796,581979.513517


In [11]:
# use the calculated NAAF factor (in totalpeaks273 dataframe, above) to caluclate the NAAF 
# NAAF: normalized normalized area abundance factor

# don't have to worry here about DECOY hits messing with Area totals
# but we will with Comet results

NAAF80 = 581979.513517

# use NAAF >80% ALC to get NAAF
peaks273_80['NAAF factor'] = (peaks273_80['NAAF num.'])/NAAF80

# make a dataframe that contains only what we need: sequences, AAs, PTMs
peaks273_AA80 = peaks273_80[['stripped peptide', 'NAAF factor', 'A', 'C', 'D', 'E', 'F', 'G', 'H', 'K', 'I/L', 'M', 'N', 'P', 'Q', 'R', 'S', 'T', 'V', 'W', 'Y', 'c-carb', 'm-oxid', 'n-deam', 'q-deam', 'k-hydr', 'p-hydr', 'r-meth']].copy()

# multiply the NAAF80 factor by the AAs to normalize its abundance by peak area and peptide length

peaks273_AA80['A-NAAF80'] = peaks273_AA80['A'] * peaks273_80['NAAF factor']
peaks273_AA80['C-NAAF80'] = peaks273_AA80['C'] * peaks273_80['NAAF factor']
peaks273_AA80['D-NAAF80'] = peaks273_AA80['D'] * peaks273_80['NAAF factor']
peaks273_AA80['E-NAAF80'] = peaks273_AA80['E'] * peaks273_80['NAAF factor']
peaks273_AA80['F-NAAF80'] = peaks273_AA80['F'] * peaks273_80['NAAF factor']
peaks273_AA80['G-NAAF80'] = peaks273_AA80['G'] * peaks273_80['NAAF factor']
peaks273_AA80['H-NAAF80'] = peaks273_AA80['H'] * peaks273_80['NAAF factor']
peaks273_AA80['K-NAAF80'] = peaks273_AA80['K'] * peaks273_80['NAAF factor']
peaks273_AA80['I/L-NAAF80'] = peaks273_AA80['I/L'] * peaks273_80['NAAF factor']
peaks273_AA80['M-NAAF80'] = peaks273_AA80['M'] * peaks273_80['NAAF factor']
peaks273_AA80['N-NAAF80'] = peaks273_AA80['N'] * peaks273_80['NAAF factor']
peaks273_AA80['P-NAAF80'] = peaks273_AA80['P'] * peaks273_80['NAAF factor']
peaks273_AA80['Q-NAAF80'] = peaks273_AA80['Q'] * peaks273_80['NAAF factor']
peaks273_AA80['R-NAAF80'] = peaks273_AA80['R'] * peaks273_80['NAAF factor']
peaks273_AA80['S-NAAF80'] = peaks273_AA80['S'] * peaks273_80['NAAF factor']
peaks273_AA80['T-NAAF80'] = peaks273_AA80['T'] * peaks273_80['NAAF factor']
peaks273_AA80['V-NAAF80'] = peaks273_AA80['V'] * peaks273_80['NAAF factor']
peaks273_AA80['W-NAAF80'] = peaks273_AA80['W'] * peaks273_80['NAAF factor']
peaks273_AA80['Y-NAAF80'] = peaks273_AA80['Y'] * peaks273_80['NAAF factor']

# multiply the NAAF80 factor by the PTMs normalize its abundance by peak area and peptide length

peaks273_AA80['ccarb-NAAF80'] = peaks273_AA80['c-carb'] * peaks273_AA80['NAAF factor']
peaks273_AA80['moxid-NAAF80'] = peaks273_AA80['m-oxid'] * peaks273_AA80['NAAF factor']
peaks273_AA80['ndeam-NAAF80'] = peaks273_AA80['n-deam'] * peaks273_AA80['NAAF factor']
peaks273_AA80['qdeam-NAAF80'] = peaks273_AA80['q-deam'] * peaks273_AA80['NAAF factor']
peaks273_AA80['khydr-NAAF80'] = peaks273_AA80['k-hydr'] * peaks273_AA80['NAAF factor']
peaks273_AA80['phydr-NAAF80'] = peaks273_AA80['p-hydr'] * peaks273_AA80['NAAF factor']
peaks273_AA80['rmeth-NAAF80'] = peaks273_AA80['r-meth'] * peaks273_AA80['NAAF factor']

# write the dataframe to a new csv
peaks273_AA80.to_csv("/home/millieginty/Documents/git-repos/2017-etnp/data/pro2020/ETNP-SKQ17/PEAKS-PTMopt/ETNP-SKQ17-273-965m-trap_PTMopt_DN80_AA_NAAF.csv")

peaks273_AA80.head()

,stripped peptide,NAAF factor,A,C,D,E,F,G,H,K,...,M-NAAF80,N-NAAF80,P-NAAF80,Q-NAAF80,R-NAAF80,S-NAAF80,T-NAAF80,V-NAAF80,W-NAAF80,Y-NAAF80
0,LLYEEEK,0.601396,0,0,0,3,0,0,0,1,...,0.0,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.0,0.000000,0.601396
1,EEDLER,2.059064,0,0,1,3,0,0,0,0,...,0.0,0.000000,0.000000,0.0,2.059064,0.0,0.000000,0.0,0.000000,0.000000
2,LLEWLK,0.265473,0,0,0,1,0,0,0,1,...,0.0,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.0,0.265473,0.000000
3,TEAPLNPK,8.870587,1,0,0,1,0,0,0,1,...,0.0,8.870587,17.741174,0.0,0.000000,0.0,8.870587,0.0,0.000000,0.000000
4,LYEETR,1.429031,0,0,0,2,0,0,0,0,...,0.0,0.000000,0.000000,0.0,1.429031,0.0,1.429031,0.0,0.000000,1.429031


### Visualizing the results

In [ ]:
print("ALC max: ", peaks['ALC (%)'].max())
print("ALC min: ", peaks['ALC (%)'].min())

In [ ]:
# take only AA totals and transpose for easier bar plotting in matplotlib

peaksaatot = totalpeaks[['A', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'K', 'L', 'M', 'N', 'P', 'Q', 'R', 'S', 'T', 'V', 'W', 'Y']].copy().T

# take only AA %s and transpose for easier bar plotting in matplotlib

peaksreltot = totalpeaks[['% C w/ carb.', '% M w/ oxid', '% N w/ deam', '% Q w/ deam', '% K w/ hydr', '% P w/ hydr', '% K w/ meth', '% R w/ meth']].copy().T

In [ ]:
# bar plot of residue totals
# there is no isoleucine (I) in Peaks data, which is why L is really big and I is 0


x_labels = ['sample total']

ax = totalpeaks.plot(y=['A', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'K', 'L', 'M', 'N', 'P', 'Q', 'R', 'S', 'T', 'V', 'W', 'Y'], kind="bar", title = '965 m trap')
plt.xticks(rotation=0)
ax.get_legend().remove()
ax.set_xticklabels(x_labels)

In [ ]:
# bar plot of residue totals
# there is no isoleucine (I) in Peaks data, which is why L is really big and I is 0

my_colors = [(x/10.0, x/20.0, 0.75) for x in range(len(peaksaatot))] # <-- Quick gradient example along the Red/Green dimensions.

ax = peaksaatot.plot(y=['sample total'], kind="bar", color = 'green', title = '965 m trap')


In [ ]:
# bar plot of relative modifications

ax = totalpeaks.plot(y=['% C w/ carb.', '% M w/ oxid', '% N w/ deam', '% Q w/ deam', '% K w/ hydr', '% P w/ hydr', '% K w/ meth', '% R w/ meth'], kind="bar", title = '965 m trap')
ax.set_xticklabels([])

In [ ]:
# bar plot of relative mods


ax = peaksreltot.plot(y=['sample total'], kind="bar", title = '965 m trap')
plt.xticks(rotation=45)

In [ ]:
# making evenly spaced bins for the ALC data based on the min and max, called above
bins = [50, 55, 60, 65, 70, 75, 80, 85, 90, 95, 100]
labels = ['50-55', '55-60', '60-65', '65-70', '70-75', '75-80', '80-85', '85-90', '90-95', '95-100']

# use pandas cut function to do the binning itself
peaks['binned'] = pd.cut(peaks['ALC (%)'], bins=bins, labels=labels)

# bar plots of binned PTM data

index = ['50-55', '55-60', '60-65', '65-70', '70-75', '75-80', '80-85', '85-90', '90-95', '95-100']
data = {'Total PTMs': [peaks.groupby('binned')['ptm-total'].sum()['50-55'], peaks.groupby('binned')['ptm-total'].sum()['55-60'], peaks.groupby('binned')['ptm-total'].sum()['60-65'], peaks.groupby('binned')['ptm-total'].sum()['65-70'], peaks.groupby('binned')['ptm-total'].sum()['70-75'], peaks.groupby('binned')['ptm-total'].sum()['75-80'], peaks.groupby('binned')['ptm-total'].sum()['80-85'], peaks.groupby('binned')['ptm-total'].sum()['85-90'], peaks.groupby('binned')['ptm-total'].sum()['90-95'], peaks.groupby('binned')['ptm-total'].sum()['95-100']],
        'Cys carb.': [peaks.groupby('binned')['c-carb'].sum()['50-55'], peaks.groupby('binned')['c-carb'].sum()['55-60'], peaks.groupby('binned')['c-carb'].sum()['60-65'], peaks.groupby('binned')['c-carb'].sum()['65-70'], peaks.groupby('binned')['c-carb'].sum()['70-75'], peaks.groupby('binned')['c-carb'].sum()['75-80'], peaks.groupby('binned')['c-carb'].sum()['80-85'], peaks.groupby('binned')['c-carb'].sum()['85-90'], peaks.groupby('binned')['c-carb'].sum()['90-95'], peaks.groupby('binned')['c-carb'].sum()['95-100']],
        'Met oxi.': [peaks.groupby('binned')['m-oxid'].sum()['50-55'], peaks.groupby('binned')['m-oxid'].sum()['55-60'], peaks.groupby('binned')['m-oxid'].sum()['60-65'], peaks.groupby('binned')['m-oxid'].sum()['65-70'], peaks.groupby('binned')['m-oxid'].sum()['70-75'], peaks.groupby('binned')['m-oxid'].sum()['75-80'], peaks.groupby('binned')['m-oxid'].sum()['80-85'], peaks.groupby('binned')['m-oxid'].sum()['85-90'], peaks.groupby('binned')['m-oxid'].sum()['90-95'], peaks.groupby('binned')['m-oxid'].sum()['95-100']],
        'Asp deam.': [peaks.groupby('binned')['n-deam'].sum()['50-55'], peaks.groupby('binned')['n-deam'].sum()['55-60'], peaks.groupby('binned')['n-deam'].sum()['60-65'], peaks.groupby('binned')['n-deam'].sum()['65-70'], peaks.groupby('binned')['n-deam'].sum()['70-75'], peaks.groupby('binned')['n-deam'].sum()['75-80'], peaks.groupby('binned')['n-deam'].sum()['80-85'], peaks.groupby('binned')['n-deam'].sum()['85-90'], peaks.groupby('binned')['n-deam'].sum()['90-95'], peaks.groupby('binned')['n-deam'].sum()['95-100']],
        'Glut deam.': [peaks.groupby('binned')['q-deam'].sum()['50-55'], peaks.groupby('binned')['q-deam'].sum()['55-60'], peaks.groupby('binned')['q-deam'].sum()['60-65'], peaks.groupby('binned')['q-deam'].sum()['65-70'], peaks.groupby('binned')['q-deam'].sum()['70-75'], peaks.groupby('binned')['q-deam'].sum()['75-80'], peaks.groupby('binned')['q-deam'].sum()['80-85'], peaks.groupby('binned')['q-deam'].sum()['85-90'], peaks.groupby('binned')['q-deam'].sum()['90-95'], peaks.groupby('binned')['q-deam'].sum()['95-100']],
        'Lys hydr': [peaks.groupby('binned')['k-hydr'].sum()['50-55'], peaks.groupby('binned')['k-hydr'].sum()['55-60'], peaks.groupby('binned')['k-hydr'].sum()['60-65'], peaks.groupby('binned')['k-hydr'].sum()['65-70'], peaks.groupby('binned')['k-hydr'].sum()['70-75'], peaks.groupby('binned')['k-hydr'].sum()['75-80'], peaks.groupby('binned')['k-hydr'].sum()['80-85'], peaks.groupby('binned')['k-hydr'].sum()['85-90'], peaks.groupby('binned')['k-hydr'].sum()['90-95'], peaks.groupby('binned')['k-hydr'].sum()['95-100']],
        'Pro hydr': [peaks.groupby('binned')['p-hydr'].sum()['50-55'], peaks.groupby('binned')['p-hydr'].sum()['55-60'], peaks.groupby('binned')['p-hydr'].sum()['60-65'], peaks.groupby('binned')['p-hydr'].sum()['65-70'], peaks.groupby('binned')['p-hydr'].sum()['70-75'], peaks.groupby('binned')['p-hydr'].sum()['75-80'], peaks.groupby('binned')['p-hydr'].sum()['80-85'], peaks.groupby('binned')['p-hydr'].sum()['85-90'], peaks.groupby('binned')['p-hydr'].sum()['90-95'], peaks.groupby('binned')['p-hydr'].sum()['95-100']],
        'Lys meth.': [peaks.groupby('binned')['k-meth'].sum()['50-55'], peaks.groupby('binned')['k-meth'].sum()['55-60'], peaks.groupby('binned')['k-meth'].sum()['60-65'], peaks.groupby('binned')['k-meth'].sum()['65-70'], peaks.groupby('binned')['k-meth'].sum()['70-75'], peaks.groupby('binned')['k-meth'].sum()['75-80'], peaks.groupby('binned')['k-meth'].sum()['80-85'], peaks.groupby('binned')['k-meth'].sum()['85-90'], peaks.groupby('binned')['k-meth'].sum()['90-95'], peaks.groupby('binned')['k-meth'].sum()['95-100']],
        'Arg meth.': [peaks.groupby('binned')['r-meth'].sum()['50-55'], peaks.groupby('binned')['r-meth'].sum()['55-60'], peaks.groupby('binned')['r-meth'].sum()['60-65'], peaks.groupby('binned')['r-meth'].sum()['65-70'], peaks.groupby('binned')['r-meth'].sum()['70-75'], peaks.groupby('binned')['r-meth'].sum()['75-80'], peaks.groupby('binned')['r-meth'].sum()['80-85'], peaks.groupby('binned')['r-meth'].sum()['85-90'], peaks.groupby('binned')['r-meth'].sum()['90-95'], peaks.groupby('binned')['r-meth'].sum()['95-100']]
        }

peaksbin = pd.DataFrame(data, columns=['Total PTMs','Cys carb.','Met oxi.','Asp deam.', 'Glut deam.', 'Lys hydr', 'Pro hydr', 'Lys meth.', 'Arg meth.'], index=index)

# write the peaks bin ptm dataframe to a csv:
peaksbin.to_csv("/home/millieginty/Documents/git-repos/2017-etnp/data/pro2020/ETNP-SKQ17/PEAKS-PTMopt/ETNP-SKQ17-273-965m-trap_DN50_ptm.csv")

ax1 = peaksbin.plot.bar(y='Total PTMs', rot=45)
ax1.set_title('Total PTMs')

ax2 = peaksbin.plot.bar(y='Cys carb.', rot=45)
ax2.set_title('Cysteine carbamidomethylation')

ax3 = peaksbin.plot.bar(y='Met oxi.', rot=45)
ax3.set_title('Methionine oxidation')

ax4 = peaksbin.plot.bar(y='Asp deam.', rot=45)
ax4.set_title('Asparagine deamidation')

ax5 = peaksbin.plot.bar(y='Glut deam.', rot=45)
ax5.set_title('Glutamine deamidation')

ax6 = peaksbin.plot.bar(y='Lys hydr', rot=45)
ax6.set_title('Lysine hydroxylation')

ax7 = peaksbin.plot.bar(y='Pro hydr', rot=45)
ax7.set_title('Proline hydroxylation')

ax8 = peaksbin.plot.bar(y='Lys meth.', rot=45)
ax8.set_title('Lysine methylation')

ax9 = peaksbin.plot.bar(y='Arg meth.', rot=45)
ax9.set_title('Arginine methylation')


In [ ]:
peaksbin['Total PTMs'] = pd.to_numeric(peaksbin['Total PTMs'])

peaksbin.loc['sums'] = peaksbin.sum(numeric_only=True, axis=0)

peaksbin.loc['rel'] = peaksbin.div(2)['sums']

print(peaksbin)

In [ ]:
# make a new dataframe of relative PTM occurance

# fist pass as numeric
peaks['ptm-total', 'c-carb'] = pd.to_numeric(peaks['ptm-total', 'c-carb'])

data = {'1': [peaks['ptm-total'].sum()], '2': [peaks['c-carb'].sum()]}
#data = {peaks[['ptm-total', 'c-carb', 'm-oxid', 'n-deam', 'q-deam', 'k-hydr', 'p-hydr', 'k-meth', 'r-meth' ]].sum()}

ptmrel = pd.DataFrame(data, columns = ['1', '2'])

ptmrel.head()

### Exporting txt files of stripped peptides at confidence cutoffs:

In [ ]:
# keep only peptide column >50% ALC
pep = peaks[["stripped peptide"]]

# write altered dataframe to new txt file
# used header and index parameters to get rid of 'Peptide' header and the indexing

pep.to_csv("/home/millieginty/Documents/git-repos/2017-etnp/data/pro2020/ETNP-SKQ17/PEAKS-PTMopt/ETNP-SKQ17-273-965m-trap_DN50_stripped_peptides.txt", header=False, index=False)


# look

print("# of DN peptide >50% ALC", len(pep))
pep.head()

In [ ]:
# keep only peptides  >80% ALC
peaks80 = peaks.loc[peaks['ALC (%)'] >= 80]

# see how many rows and double check
# peaks80.head(-10)

# keep only peptide column 
pep80 = peaks80[["stripped peptide"]]

# write altered dataframe to new txt file
# used header and index parameters to get rid of 'Peptide' header and the indexing

pep80.to_csv("/home/millieginty/Documents/git-repos/2017-etnp/data/pro2020/ETNP-SKQ17/PEAKS-PTMopt/ETNP-SKQ17-273-965m-trap_DN80_stripped_peptides.txt", header=False, index=False)


print("# of DN peptide >80% ALC", len(pep80))
pep80.head()